In [ ]:
import pandas as pd   # ver.1 заниженные прогнозы
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# Загрузка данных о продажах из Excel
df = pd.read_excel("sales.xlsx")
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y")

# Загрузка данных о ценах из Excel
prices_df = pd.read_excel("new_prices.xlsx")

# Получение последней даты в исходных данных
last_date = df['Дата'].max()

# Создание датафрейма с данными о ценах на все 14 дней
forecast_dates = [last_date + timedelta(days=i) for i in range(1, 15)]
forecast_data = pd.DataFrame({'Дата': forecast_dates})

# Дублируем данные о ценах для каждой номенклатуры
forecast_data['Цена'] = prices_df['Цена'].iloc[0]

# Разделение данных на признаки и целевую переменную
X = df[['Цена']]
y = df['Кол-во продано, шт.']

# Обучение модели случайного леса для каждой номенклатуры
models = {}
for item in df['Номенклатура'].unique():
    item_df = df[df['Номенклатура'] == item]
    X_item = item_df[['Цена']]
    y_item = item_df['Кол-во продано, шт.']
    model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_item, y_item)
    models[item] = model

# Создание списка для хранения суммарных прогнозов
total_forecast_list = []

# Прогноз продаж для каждой номенклатуры
for item, model in models.items():
    forecast_sales = model.predict(forecast_data[['Цена']])
    total_forecast_sales = sum(forecast_sales)
    total_forecast_list.append((item, total_forecast_sales))

# Создание датафрейма с суммарными прогнозами
total_forecast_df = pd.DataFrame(total_forecast_list, columns=['Номенклатура', 'Суммарный прогноз'])

# Сохранение результатов прогноза в Excel
total_forecast_df.to_excel("прогноз_продаж.xlsx", index=False)

print("Прогноз продаж сохранен в файл 'прогноз_продаж.xlsx'")


In [ ]:
import pandas as pd
import lightgbm as lgb
from datetime import datetime, timedelta

# Загрузка данных о продажах из Excel
df = pd.read_excel("sales.xlsx")
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y")

# Загрузка данных о ценах из Excel
prices_df = pd.read_excel("new_prices.xlsx")

# Получение последней даты в исходных данных
last_date = df['Дата'].max()

# Создание датафрейма с данными о ценах на все 14 дней
forecast_dates = [last_date + timedelta(days=i) for i in range(1, 15)]
forecast_data = pd.DataFrame({'Дата': forecast_dates})

# Дублируем данные о ценах для каждой номенклатуры
forecast_data['Цена'] = prices_df['Цена'].iloc[0]

# Разделение данных на признаки и целевую переменную
X = df[['Цена']]
y = df['Кол-во продано, шт.']

# Обучение модели LightGBM для каждой номенклатуры
models = {}
for item in df['Номенклатура'].unique():
    item_df = df[df['Номенклатура'] == item]
    X_item = item_df[['Цена']]
    y_item = item_df['Кол-во продано, шт.']
    lgb_train = lgb.Dataset(X_item, y_item)
    params = {'objective': 'regression',
              'metric': 'mse',
              'num_leaves': 30,
              'learning_rate': 0.01}
    model = lgb.train(params, lgb_train, num_boost_round=100)
    models[item] = model

# Создание списка для хранения суммарных прогнозов
total_forecast_list = []

# Прогноз продаж для каждой номенклатуры
for item, model in models.items():
    forecast_sales = model.predict(forecast_data[['Цена']])
    total_forecast_sales = sum(forecast_sales)
    total_forecast_list.append((item, total_forecast_sales))

# Создание датафрейма с суммарными прогнозами
total_forecast_df = pd.DataFrame(total_forecast_list, columns=['Номенклатура', 'Суммарный прогноз'])

# Сохранение результатов прогноза в Excel
total_forecast_df.to_excel("прогноз_промо_продаж.xlsx", index=False)

print("Прогноз продаж сохранен в файл 'прогноз_промо_продаж.xlsx'")


# SARIMAx

In [ ]:
%%time                                 #ТОПЧИК Без минусовых прогнозов(средние за 30 дней)
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Загрузка данных из Excel
sales_data = pd.read_excel('Продажи.xlsx', sheet_name='Продажи')
price_data = pd.read_excel('Продажи.xlsx', sheet_name='Цены')

# Подготовка данных
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], format="%d.%m.%Y")
sales_data.set_index('Дата', inplace=True)

# Создание списка уникальных товаров
unique_items = sales_data['Номенклатура'].unique()

# Прогнозирование и сохранение результатов для каждого товара
forecast_results = []

for item in unique_items:
    # Фильтрация данных для текущего товара
    item_sales_data = sales_data[sales_data['Номенклатура'] == item]

    if len(item_sales_data) < 30:
        # Пропускаем товары с недостаточным количеством данных для прогноза
        continue

    # Последняя известная цена для текущего товара
    last_price = price_data[price_data['Номенклатура'] == item]['Цена'].iloc[-1]

    # Повторение цены для каждого дня прогноза
    exog_forecast = np.full((30, 1), last_price)

    try:
        # Построение модели SARIMA для текущего товара с учетом последней известной цены
        model = SARIMAX(item_sales_data['Кол-во продано, шт.'], exog=item_sales_data['Цена'], order=(1, 0, 1), seasonal_order=(1, 0, 1, 7))
        result = model.fit()

        # Прогноз на 30 дней вперед, используя последнюю известную цену
        forecast = result.get_forecast(steps=30, exog=exog_forecast)
        predicted_mean = forecast.predicted_mean

        # Вычисление среднего значения продаж за последние 30 дней
        last_30_days_sales = item_sales_data['Кол-во продано, шт.'].tail(30)
        mean_sales_last_30_days = last_30_days_sales.mean()

        # Замена отрицательных прогнозов на средние продажи за последние 30 дней
        predicted_mean = predicted_mean.apply(lambda x: mean_sales_last_30_days if x < 0 else x)

        total_forecast = predicted_mean.sum()

        # Добавление результатов прогноза для текущего товара в список
        forecast_results.append({'Код Товара': item, 'Прогноз': total_forecast})
    except:
        # Пропускаем товары, для которых не удалось построить модель SARIMA
        continue

# Создание DataFrame с результатами прогноза
forecast_df = pd.DataFrame(forecast_results)

# Сохранение результатов в Excel
forecast_df.to_excel('прогноз_продаж_по_товарамSARIMA.xlsx', index=False)

print("Прогноз продаж по каждому товару сохранен в файл 'прогноз_продаж_по_товарамSARIMA.xlsx'")


# Stacking

In [2]:
import pandas as pd                                            # ПУШКА СТЭКИНГ БАМ! БАМ!
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.linear_model import LinearRegression

# Загрузка данных из Excel
sales_data = pd.read_excel('Продажи.xlsx', sheet_name='Продажи')
price_data = pd.read_excel('Продажи.xlsx', sheet_name='Цены')

# Подготовка данных
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], format="%d.%m.%Y")
sales_data.set_index('Дата', inplace=True)

# Создание списка уникальных товаров
unique_items = sales_data['Номенклатура'].unique()

# Прогнозирование и сохранение результатов для каждого товара
forecast_results = []

for item in unique_items:
    # Фильтрация данных для текущего товара
    item_sales_data = sales_data[sales_data['Номенклатура'] == item]

    if len(item_sales_data) < 30:
        # Пропускаем товары с недостаточным количеством данных для прогноза
        continue

    # Последняя известная цена для текущего товара
    last_price = price_data[price_data['Номенклатура'] == item]['Цена'].iloc[-1]
    exog_forecast = np.full((30, 1), last_price)

    try:
        # Модель SARIMA
        sarima_model = SARIMAX(item_sales_data['Кол-во продано, шт.'], exog=item_sales_data['Цена'], order=(1, 0, 1), seasonal_order=(1, 0, 1, 7))
        sarima_result = sarima_model.fit()
        sarima_forecast = sarima_result.get_forecast(steps=30, exog=exog_forecast)
        sarima_predicted = sarima_forecast.predicted_mean

        # Модель Prophet
        prophet_data = item_sales_data.reset_index().rename(columns={'Дата': 'ds', 'Кол-во продано, шт.': 'y'})
        prophet_model = Prophet()
        prophet_model.fit(prophet_data)
        future = prophet_model.make_future_dataframe(periods=30)
        prophet_forecast = prophet_model.predict(future)
        prophet_predicted = prophet_forecast['yhat'].tail(30)

        # Обучение метамодели (линейная регрессия)
        X_train = np.column_stack([sarima_predicted, prophet_predicted])
        y_train = item_sales_data['Кол-во продано, шт.'].tail(30).values  # истинные значения продаж

        meta_model = LinearRegression()
        meta_model.fit(X_train, y_train)

        # Прогнозирование на основе метамодели
        X_test = np.column_stack([sarima_predicted, prophet_predicted])
        final_forecast = meta_model.predict(X_test)

        # Замена отрицательных прогнозов на средние продажи за последние 30 дней
        last_30_days_sales = item_sales_data['Кол-во продано, шт.'].tail(30)
        mean_sales_last_30_days = last_30_days_sales.mean()
        final_forecast = np.where(final_forecast < 0, mean_sales_last_30_days, final_forecast)

        total_forecast = final_forecast.sum()

        # Добавление результатов прогноза для текущего товара в список
        forecast_results.append({'Код Товара': item, 'Прогноз': total_forecast})

    except Exception as e:
        print(f"Ошибка для товара {item}: {e}")
        continue

# Создание DataFrame с результатами прогноза
forecast_df = pd.DataFrame(forecast_results)

# Сохранение результатов в Excel
forecast_df.to_excel('прогноз_продаж_по_товарам_stacking.xlsx', index=False)

print("Прогноз продаж по каждому товару сохранен в файл 'прогноз_продаж_по_товарам_stacking.xlsx'")


C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
15:45:44 - cmdstanpy - INFO - 

Ошибка для товара 88343: LU decomposition error.


15:46:28 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
15:46:29 - cmdstanpy - INFO - Chain [1] start processing
15:46:29 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\bondarenkovv\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: Va

Прогноз продаж по каждому товару сохранен в файл 'прогноз_продаж_по_товарам_stacking.xlsx'
